## Subsetting ICESat-2 Data with the NSIDC Subsetter
### How to Use the NSIDC Subsetter Example Notebook
This notebook illustrates the use of `icepyx` for subsetting ICESat-2 data ordered through the NSIDC DAAC. We'll show how to find out what subsetting options are available and how to specify the subsetting options for your order.

For more information on using `icepyx` to find, order, and download data, see the [02-Data_Access Tutorial](https://github.com/ICESAT-2HackWeek/data-access/blob/master/notebooks/02-Data_Access.ipynb) or `icepyx`'s complimentary [ICESat-2_DAAC_DataAccess_Example Notebook](https://github.com/icesat2py/icepyx/blob/master/doc/examples/ICESat-2_DAAC_DataAccess_Example.ipynb).

Questions? Be sure to check out the FAQs throughout this notebook, indicated as italic headings.

#### Credits
* tutorial notebook by: Jessica Scheick and Amy Steiker
* original notebook by: Jessica Scheick and Zheng Liu
* some source material: [NSIDC Data Access Notebook](https://github.com/ICESAT-2HackWeek/ICESat2_hackweek_tutorials/tree/master/03_NSIDCDataAccess_Steiker) by Amy Steiker and Bruce Wallin

**Please note that the tutorials presented here used version 0.2.0 of `icepyx`. These tutorials are set up to use that version in the provided Binder link, but they will not run without edits on more recent versions (>= v0.3.0) of `icepyx` due to non-backwards compatible changes in syntax. Similar, up-to-date data access tutorials/examples are available [from `icepyx`](https://icepyx.readthedocs.io/en/latest/getting_started/example_link.html).**

### _What is SUBSETTING anyway?_

Anyone who's worked with geospatial data has probably encountered subsetting. Typically, we search for data wherever it is stored and download the chunks (aka granules, scenes, passes, swaths, etc.) that contain something we are interested in. Then, we have to extract from each chunk the pieces we actually want to analyze. Those pieces might be geospatial (i.e. an area of interest), temporal (i.e. certain months of a time series), and/or certain variables. This process of extracting the data we are going to use is called subsetting.

In the case of ICESat-2 data coming from the NSIDC DAAC, we can do this subsetting step on the data prior to download, reducing our number of data processing steps and resulting in smaller, faster downloads and storage.

### Import packages, including icepyx

In [1]:
from icepyx import icesat2data as ipd

import numpy as np
import xarray as xr
import pandas as pd

import h5py
import os,json
from pprint import pprint

### Create an icesat2data object and log in to Earthdata

For this example, we'll be working with a land ice dataset (ATL06) for an area along West Greenland (Disko Bay). A second option for an atmospheric product that uses profiles instead of the ground track (gt) categorization is also provided.

In [2]:
region_a = ipd.Icesat2Data('ATL06',[-55, 68, -48, 71],['2019-02-22','2019-02-28'], \
                           start_time='00:00:00', end_time='23:59:59')

In [3]:
# Uncomment and run the code in this cell to use the second variable subsetting suite of examples,
# with the beam specifier containing "profile" instead of "gt#l"

# region_a = ipd.Icesat2Data('ATL09',[-55, 68, -48, 71],['2019-02-22','2019-02-28'], \
#              start_time='00:00:00', end_time='23:59:59')

In [ ]:
region_a.earthdata_login('icepyx_devteam','icepyx.dev@gmail.com')

### Discover Subsetting Options

You can see what subsetting options are available for a given dataset by calling `show_custom_options()`. The options are presented as a series of headings followed by available values in square brackets. Headings are:
* **Subsetting Options**: whether or not temporal and spatial subsetting are available for the dataset
* **Data File Formats (Reformatting Options)**: return the data in a format other than the native hdf5 (submitted as a key=value kwarg to `order_granules(format='NetCDF4-CF')`)
* **Data File (Reformatting) Options Supporting Reprojection**: return the data in a reprojected reference frame. These options will be supported in the future for gridded ICESat-2 L3B datasets, once those are produced and made available.
* **Data File (Reformatting) Options NOT Supporting Reprojection**: data file formats that cannot be delivered with reprojection
* **Data Variables (also Subsettable)**: a dictionary of variable name keys and the paths to those variables available in the dataset

In [6]:
region_a.show_custom_options(dictview=True)

Subsetting options
[{'id': 'ICESAT2',
  'maxGransAsyncRequest': '2000',
  'maxGransSyncRequest': '100',
  'spatialSubsetting': 'true',
  'spatialSubsettingShapefile': 'true',
  'temporalSubsetting': 'true',
  'type': 'both'}]
Data File Formats (Reformatting Options)
['TABULAR_ASCII', 'NetCDF4-CF', 'Shapefile', 'NetCDF-3']
Reprojection Options
[]
Data File (Reformatting) Options Supporting Reprojection
['TABULAR_ASCII', 'NetCDF4-CF', 'Shapefile', 'NetCDF-3', 'No reformatting']
Data File (Reformatting) Options NOT Supporting Reprojection
[]
Data Variables (also Subsettable)
{'atl06_quality_summary': ['gt1l/land_ice_segments/atl06_quality_summary',
                           'gt1r/land_ice_segments/atl06_quality_summary',
                           'gt2l/land_ice_segments/atl06_quality_summary',
                           'gt2r/land_ice_segments/atl06_quality_summary',
                           'gt3l/land_ice_segments/atl06_quality_summary',
                           'gt3r/land_ice_segm

By default, spatial and temporal subsetting based on your initial inputs is applied to your order unless you specify `subset=False` to `order_granules()` or `download_granules()` (which calls `order_granules` under the hood if you have not already placed your order) functions. Additional subsetting options must be specified as keyword arguments to the order/download functions.

Although some file format conversions and reprojections are possible using the `format`, `projection`, and `projection_parameters` keywords, the rest of this tutorial will focus on variable subsetting, which is provided with the `Coverage` keyword. Note that as of June 2020, cryospheric ICESat-2 L3B datasets are not yet available, therefore no projection options currently exist.

### _Why do I have to provide spatial bounds to icepyx even if I don't use them to subset my data order?_

Because they're still needed for the granule level search. Spatial inputs are usually required for any data search, on any platform, even if your search parameters cover the entire globe.

The spatial information you provide is used to search the data repository and determine which granules might contain data over your area of interest. When you use that spatial information for subsetting, it's actually asking the NSIDC subsetter to extract the appropriate data from each granule. Thus, even if you set `subset=False` and download entire granules, you still need to provide some inputs on what geographic area you'd like data for.

## About Data Variables in an Icesat2Data object
There are two possible variable parameters associated with each ```icesat2data``` object.
1. `order_vars`, which is for interacting with variables during data querying, ordering, and downloading activities. `order_vars.wanted` holds the user's list to be submitted to the NSIDC subsetter and download a smaller, reproducible dataset.
2. `file_vars`, which is for interacting with variables associated with local files [not yet implemented].

Each variables parameter (which is actually an associated Variables class object) has methods to:
* get available variables, either available from the NSIDC or the file (`avail()` method/attribute).
* append new variables to the wanted list (`append()` method).
* remove variables from the wanted list (`remove()` method).

Each variables instance also has a set of attributes, including `avail` and `wanted` to indicate the list of variables that is available (unmutable, or unchangeable, as it is based on the input dataset specifications or files) and the list of variables that the user would like extracted (updateable with the `append` and `remove` methods), respectively. We'll showcase the use of all of these methods and attributes below.

### ICESat-2 data variables
ICESat-2 data is natively stored in a nested file format called hdf5. Much like a directory-file system on a computer, each variable (file) has a unique path through the heirarchy (directories) within the file. Thus, some variables (e.g. `'latitude'`, `'longitude'`) have multiple paths (one for each of the six beams in most datasets). 

To increase readability, some display options (2 and 3, below) show the 200+ variable + path combinations as a dictionary where the keys are variable names and the values are the paths to that variable.

### Determine what variables are available
There are multiple ways to get a complete list of available variables.

1. `region_a.order_vars.avail`, a list of all valid path+variable strings
2. `region_a.show_custom_options(dictview=True)`, all available subsetting and data transformation options, as you ran above
3. `region_a.order_vars.parse_var_list(region_a.order_vars.avail)`, a dictionary of variable:paths key:value pairs

In [7]:
region_a.order_vars.avail()

['ancillary_data/atlas_sdp_gps_epoch',
 'ancillary_data/control',
 'ancillary_data/data_end_utc',
 'ancillary_data/data_start_utc',
 'ancillary_data/end_cycle',
 'ancillary_data/end_delta_time',
 'ancillary_data/end_geoseg',
 'ancillary_data/end_gpssow',
 'ancillary_data/end_gpsweek',
 'ancillary_data/end_orbit',
 'ancillary_data/end_region',
 'ancillary_data/end_rgt',
 'ancillary_data/granule_end_utc',
 'ancillary_data/granule_start_utc',
 'ancillary_data/qa_at_interval',
 'ancillary_data/release',
 'ancillary_data/start_cycle',
 'ancillary_data/start_delta_time',
 'ancillary_data/start_geoseg',
 'ancillary_data/start_gpssow',
 'ancillary_data/start_gpsweek',
 'ancillary_data/start_orbit',
 'ancillary_data/start_region',
 'ancillary_data/start_rgt',
 'ancillary_data/version',
 'ancillary_data/land_ice/dt_hist',
 'ancillary_data/land_ice/fit_maxiter',
 'ancillary_data/land_ice/fpb_maxiter',
 'ancillary_data/land_ice/maxiter',
 'ancillary_data/land_ice/max_res_ids',
 'ancillary_data/lan

By passing the boolean `options=True` to the `avail` method, you can obtain lists of unique possible variable inputs (var_list inputs) and path subdirectory inputs (keyword_list and beam_list inputs) for your dataset. These can be helpful for building your wanted variable list.

In [8]:
region_a.order_vars.avail(options=True)

var_list inputs: atlas_sdp_gps_epoch, control, data_end_utc, data_start_utc, end_cycle, end_delta_time, end_geoseg, end_gpssow, end_gpsweek, end_orbit, end_region, end_rgt, granule_end_utc, granule_start_utc, qa_at_interval, release, start_cycle, start_delta_time, start_geoseg, start_gpssow, start_gpsweek, start_orbit, start_region, start_rgt, version, dt_hist, fit_maxiter, fpb_maxiter, maxiter, max_res_ids, min_dist, min_gain_th, min_n_pe, min_n_sel, min_signal_conf, n_hist, nhist_bins, n_sigmas, proc_interval, qs_lim_bsc, qs_lim_hrs, qs_lim_hsigma, qs_lim_msw, qs_lim_snr, qs_lim_sss, rbin_width, sigma_beam, sigma_tx, t_dead, atl06_quality_summary, delta_time, h_li, h_li_sigma, latitude, longitude, segment_id, sigma_geo_h, fpb_mean_corr, fpb_mean_corr_sigma, fpb_med_corr, fpb_med_corr_sigma, fpb_n_corr, med_r_fit, tx_mean_corr, tx_med_corr, dem_flag, dem_h, geoid_h, dh_fit_dx, dh_fit_dx_sigma, dh_fit_dy, h_expected_rms, h_mean, h_rms_misfit, h_robust_sprd, n_fit_photons, n_seg_pulses,

### _Why not just download all the data and subset locally? What if I need more variables/granules?_

Taking advantage of the NSIDC subsetter is a great way to reduce your download size and thus your download time and the amount of storage required, especially if you're storing your data locally during analysis. By downloading your data using `icepyx`, it is easy to go back and get additional data with the same, similar, or different parameters (e.g. you can keep the same spatial and temporal bounds but change the variable list). Related tools (e.g. [`captoolkit`](https://github.com/fspaolo/captoolkit)) will let you easily merge files if you're uncomfortable merging them during read-in for processing.

### Building your wanted variable list

Now that you know which variables and path components are available for your dataset, you need to build a list of the ones you'd like included in your dataset. There are several options for generating your initial list as well as modifying it, giving the user complete control over the list submitted.

The options for building your initial list are:
1. Use a default list for the dataset with the `defaults=True` keyword argument (not yet fully implemented across all datasets. Have a default variable list for your field/dataset? Submit a pull request or post it as an issue on [GitHub](https://github.com/icesat2py/icepyx)!)
2. Provide a list of variable names
3. Provide a list of profiles/beams or other path keywords, where "keywords" are simply the unique subdirectory names contained in the full variable paths of the dataset. A full list of available keywords for the dataset is displayed in the error message upon entering `keyword_list=['']` into the `append` function (see below for an example) or by running `region_a.order_vars.avail(options=True)`, as above

**Note: all datasets have a short list of "mandatory" variables/paths (containing spacecraft orientation and time information needed to convert the data's `delta_time` to a readable datetime) that are automatically added to any built list. If you have any recommendations for other variables that should always be included (e.g. uncertainty information), please let us know!**

Examples of using each method to build and modify your wanted variable list are below.

In [9]:
region_a.order_vars.wanted

In [10]:
region_a.order_vars.append(defaults=True)
pprint(region_a.order_vars.wanted)

{'atl06_quality_summary': ['gt1l/land_ice_segments/atl06_quality_summary',
                           'gt1r/land_ice_segments/atl06_quality_summary',
                           'gt2l/land_ice_segments/atl06_quality_summary',
                           'gt2r/land_ice_segments/atl06_quality_summary',
                           'gt3l/land_ice_segments/atl06_quality_summary',
                           'gt3r/land_ice_segments/atl06_quality_summary'],
 'atlas_sdp_gps_epoch': ['ancillary_data/atlas_sdp_gps_epoch'],
 'bsnow_conf': ['gt1l/land_ice_segments/geophysical/bsnow_conf',
                'gt1r/land_ice_segments/geophysical/bsnow_conf',
                'gt2l/land_ice_segments/geophysical/bsnow_conf',
                'gt2r/land_ice_segments/geophysical/bsnow_conf',
                'gt3l/land_ice_segments/geophysical/bsnow_conf',
                'gt3r/land_ice_segments/geophysical/bsnow_conf'],
 'bsnow_h': ['gt1l/land_ice_segments/geophysical/bsnow_h',
             'gt1r/land_ice_segment

Reminder: the keywords available for this dataset are shown in the error message upon entering a blank keyword_list, as seen in the next cell.

In [11]:
region_a.order_vars.append(keyword_list=[''])

ValueError: Invalid keyword: . Please select from this list: ancillary_data, bias_correction, dem, fit_statistics, geophysical, ground_track, gt1l, gt1r, gt2l, gt2r, gt3l, gt3r, land_ice, land_ice_segments, none, orbit_info, quality_assessment, residual_histogram, segment_quality, signal_selection_status

### Modifying your wanted variable list

Generating and modifying your variable request list, which is stored in `region_a.order_vars.wanted`, is controlled by the `append` and `remove` functions that operate on `region_a.order_vars.wanted`. The input options to `append` are as follows (the full documentation for this function can be found by executing `help(region_a.order_vars.append)`).
* `defaults` (default False) - include the default variable list for your dataset (not yet fully implemented for all datasets; please submit your default variable list for inclusion!)
* `var_list` (default None) - list of variables (entered as strings)
* `beam_list` (default None) - list of beams/profiles (entered as strings)
* `keyword_list` (default None) - list of keywords (entered as strings); use `keyword_list=['']` to obtain a list of available keywords

Similarly, the options for `remove` are:
* `all` (default False) - reset `region_a.order_vars.wanted` to None
* `var_list` (as above)
* `beam_list` (as above)
* `keyword_list` (as above)

In [12]:
region_a.order_vars.remove(all=True)
pprint(region_a.order_vars.wanted)

None


### Examples (Overview)

Below are a series of examples to show how you can use `append` and `remove` to modify your wanted variable list. For clarity, `region_a.order_vars.wanted` is cleared at the start of many examples. However, multiple `append` and `remove` commands can be called in succession to build your wanted variable list (see Examples 3+)

There are two example tracks. The first is for land ice (ATL06) data that is separated into beams. The second is for atmospheric data (ATL09) that is separated into profiles. Both example tracks showcase the same functionality and are provided for users of both data types.

------------------
### Example Track 1 (Land Ice - run with ATL06 dataset)

#### Example 1: choose variables
Add all `latitude` and `longitude` variables across all six beam groups. Note that the additional required variables for time and spacecraft orientation are included by default.

In [13]:
region_a.order_vars.append(var_list=['latitude','longitude'])
pprint(region_a.order_vars.wanted)

{'atlas_sdp_gps_epoch': ['ancillary_data/atlas_sdp_gps_epoch'],
 'data_end_utc': ['ancillary_data/data_end_utc'],
 'data_start_utc': ['ancillary_data/data_start_utc'],
 'end_delta_time': ['ancillary_data/end_delta_time'],
 'granule_end_utc': ['ancillary_data/granule_end_utc'],
 'granule_start_utc': ['ancillary_data/granule_start_utc'],
 'latitude': ['gt1l/land_ice_segments/latitude',
              'gt1r/land_ice_segments/latitude',
              'gt2l/land_ice_segments/latitude',
              'gt2r/land_ice_segments/latitude',
              'gt3l/land_ice_segments/latitude',
              'gt3r/land_ice_segments/latitude'],
 'longitude': ['gt1l/land_ice_segments/longitude',
               'gt1r/land_ice_segments/longitude',
               'gt2l/land_ice_segments/longitude',
               'gt2r/land_ice_segments/longitude',
               'gt3l/land_ice_segments/longitude',
               'gt3r/land_ice_segments/longitude'],
 'sc_orient': ['orbit_info/sc_orient'],
 'sc_orient_time': [

#### Example 2: specify beams and variable
Add `latitude` for only `gt1l` and `gt2l`

In [14]:
region_a.order_vars.remove(all=True)
pprint(region_a.order_vars.wanted)

None


In [15]:
var_dict = region_a.order_vars.append(beam_list=['gt1l', 'gt2l'], var_list=['latitude'])
pprint(region_a.order_vars.wanted)

{'atlas_sdp_gps_epoch': ['ancillary_data/atlas_sdp_gps_epoch'],
 'data_end_utc': ['ancillary_data/data_end_utc'],
 'data_start_utc': ['ancillary_data/data_start_utc'],
 'end_delta_time': ['ancillary_data/end_delta_time'],
 'granule_end_utc': ['ancillary_data/granule_end_utc'],
 'granule_start_utc': ['ancillary_data/granule_start_utc'],
 'latitude': ['gt1l/land_ice_segments/latitude',
              'gt2l/land_ice_segments/latitude'],
 'sc_orient': ['orbit_info/sc_orient'],
 'sc_orient_time': ['orbit_info/sc_orient_time'],
 'start_delta_time': ['ancillary_data/start_delta_time']}


#### Example 3: add/remove selected beams+variables
Add `latitude` for `gt3l` and remove it for `gt2l`

In [16]:
region_a.order_vars.append(beam_list=['gt3l'],var_list=['latitude'])
region_a.order_vars.remove(beam_list=['gt2l'], var_list=['latitude'])
pprint(region_a.order_vars.wanted)

{'atlas_sdp_gps_epoch': ['ancillary_data/atlas_sdp_gps_epoch'],
 'data_end_utc': ['ancillary_data/data_end_utc'],
 'data_start_utc': ['ancillary_data/data_start_utc'],
 'end_delta_time': ['ancillary_data/end_delta_time'],
 'granule_end_utc': ['ancillary_data/granule_end_utc'],
 'granule_start_utc': ['ancillary_data/granule_start_utc'],
 'latitude': ['gt1l/land_ice_segments/latitude',
              'gt3l/land_ice_segments/latitude'],
 'sc_orient': ['orbit_info/sc_orient'],
 'sc_orient_time': ['orbit_info/sc_orient_time'],
 'start_delta_time': ['ancillary_data/start_delta_time']}


#### Example 4: `keyword_list`
Add `latitude` and `longitude` for all beams and with keyword `land_ice_segments`

In [17]:
region_a.order_vars.append(var_list=['latitude', 'longitude'],keyword_list=['land_ice_segments'])
pprint(region_a.order_vars.wanted)

{'atlas_sdp_gps_epoch': ['ancillary_data/atlas_sdp_gps_epoch'],
 'data_end_utc': ['ancillary_data/data_end_utc'],
 'data_start_utc': ['ancillary_data/data_start_utc'],
 'end_delta_time': ['ancillary_data/end_delta_time'],
 'granule_end_utc': ['ancillary_data/granule_end_utc'],
 'granule_start_utc': ['ancillary_data/granule_start_utc'],
 'latitude': ['gt1l/land_ice_segments/latitude',
              'gt3l/land_ice_segments/latitude',
              'gt1r/land_ice_segments/latitude',
              'gt2l/land_ice_segments/latitude',
              'gt2r/land_ice_segments/latitude',
              'gt3r/land_ice_segments/latitude'],
 'longitude': ['gt1l/land_ice_segments/longitude',
               'gt1r/land_ice_segments/longitude',
               'gt2l/land_ice_segments/longitude',
               'gt2r/land_ice_segments/longitude',
               'gt3l/land_ice_segments/longitude',
               'gt3r/land_ice_segments/longitude'],
 'sc_orient': ['orbit_info/sc_orient'],
 'sc_orient_time': [

#### Example 5: target a specific variable + path
Remove `gt1r/land_ice_segments/longitude` (but keep `gt1r/land_ice_segments/latitude`)

In [18]:
region_a.order_vars.remove(beam_list=['gt1r'], var_list=['longitude'], keyword_list=['land_ice_segments'])
pprint(region_a.order_vars.wanted)

{'atlas_sdp_gps_epoch': ['ancillary_data/atlas_sdp_gps_epoch'],
 'data_end_utc': ['ancillary_data/data_end_utc'],
 'data_start_utc': ['ancillary_data/data_start_utc'],
 'end_delta_time': ['ancillary_data/end_delta_time'],
 'granule_end_utc': ['ancillary_data/granule_end_utc'],
 'granule_start_utc': ['ancillary_data/granule_start_utc'],
 'latitude': ['gt1l/land_ice_segments/latitude',
              'gt3l/land_ice_segments/latitude',
              'gt1r/land_ice_segments/latitude',
              'gt2l/land_ice_segments/latitude',
              'gt2r/land_ice_segments/latitude',
              'gt3r/land_ice_segments/latitude'],
 'longitude': ['gt1l/land_ice_segments/longitude',
               'gt2l/land_ice_segments/longitude',
               'gt2r/land_ice_segments/longitude',
               'gt3l/land_ice_segments/longitude',
               'gt3r/land_ice_segments/longitude'],
 'sc_orient': ['orbit_info/sc_orient'],
 'sc_orient_time': ['orbit_info/sc_orient_time'],
 'start_delta_time': 

#### Example 6: add variables not specific to beams/profiles
Add `rgt` under `orbit_info`.

In [19]:
region_a.order_vars.append(keyword_list=['orbit_info'],var_list=['rgt'])
pprint(region_a.order_vars.wanted)

{'atlas_sdp_gps_epoch': ['ancillary_data/atlas_sdp_gps_epoch'],
 'data_end_utc': ['ancillary_data/data_end_utc'],
 'data_start_utc': ['ancillary_data/data_start_utc'],
 'end_delta_time': ['ancillary_data/end_delta_time'],
 'granule_end_utc': ['ancillary_data/granule_end_utc'],
 'granule_start_utc': ['ancillary_data/granule_start_utc'],
 'latitude': ['gt1l/land_ice_segments/latitude',
              'gt3l/land_ice_segments/latitude',
              'gt1r/land_ice_segments/latitude',
              'gt2l/land_ice_segments/latitude',
              'gt2r/land_ice_segments/latitude',
              'gt3r/land_ice_segments/latitude'],
 'longitude': ['gt1l/land_ice_segments/longitude',
               'gt2l/land_ice_segments/longitude',
               'gt2r/land_ice_segments/longitude',
               'gt3l/land_ice_segments/longitude',
               'gt3r/land_ice_segments/longitude'],
 'rgt': ['orbit_info/rgt'],
 'sc_orient': ['orbit_info/sc_orient'],
 'sc_orient_time': ['orbit_info/sc_orient_t

#### Example 7: add all variables+paths of a group
In addition to adding specific variables and paths, we can filter all variables with a specific keyword as well. Here, we add all variables under `orbit_info`. Note that paths already in `region_a.order_vars.wanted`, such as `'orbit_info/rgt'`, are not duplicated.

In [20]:
region_a.order_vars.append(keyword_list=['orbit_info'])
pprint(region_a.order_vars.wanted)

{'atlas_sdp_gps_epoch': ['ancillary_data/atlas_sdp_gps_epoch'],
 'crossing_time': ['orbit_info/crossing_time'],
 'cycle_number': ['orbit_info/cycle_number'],
 'data_end_utc': ['ancillary_data/data_end_utc'],
 'data_start_utc': ['ancillary_data/data_start_utc'],
 'end_delta_time': ['ancillary_data/end_delta_time'],
 'granule_end_utc': ['ancillary_data/granule_end_utc'],
 'granule_start_utc': ['ancillary_data/granule_start_utc'],
 'lan': ['orbit_info/lan'],
 'latitude': ['gt1l/land_ice_segments/latitude',
              'gt3l/land_ice_segments/latitude',
              'gt1r/land_ice_segments/latitude',
              'gt2l/land_ice_segments/latitude',
              'gt2r/land_ice_segments/latitude',
              'gt3r/land_ice_segments/latitude'],
 'longitude': ['gt1l/land_ice_segments/longitude',
               'gt2l/land_ice_segments/longitude',
               'gt2r/land_ice_segments/longitude',
               'gt3l/land_ice_segments/longitude',
               'gt3r/land_ice_segments/lo

#### Example 8: add all possible values for variables+paths
Append all `longitude` paths and all variables/paths with keyword `land_ice_segments`.

Similarly to what is shown in Example 4, if you submit only one `append` call as `region_a.order_vars.append(var_list=['longitude'], keyword_list=['land_ice_segments'])` rather than the two `append` calls shown below, you will only add the variable `longitude` and only paths containing `land_ice_segments`, not ALL paths for `longitude` and ANY variables with `land_ice_segments` in their path.

In [21]:
region_a.order_vars.append(var_list=['longitude'])
region_a.order_vars.append(keyword_list=['land_ice_segments'])
pprint(region_a.order_vars.wanted)

{'atl06_quality_summary': ['gt1l/land_ice_segments/atl06_quality_summary',
                           'gt1r/land_ice_segments/atl06_quality_summary',
                           'gt2l/land_ice_segments/atl06_quality_summary',
                           'gt2r/land_ice_segments/atl06_quality_summary',
                           'gt3l/land_ice_segments/atl06_quality_summary',
                           'gt3r/land_ice_segments/atl06_quality_summary'],
 'atlas_sdp_gps_epoch': ['ancillary_data/atlas_sdp_gps_epoch'],
 'bckgrd': ['gt1l/land_ice_segments/geophysical/bckgrd',
            'gt1r/land_ice_segments/geophysical/bckgrd',
            'gt2l/land_ice_segments/geophysical/bckgrd',
            'gt2r/land_ice_segments/geophysical/bckgrd',
            'gt3l/land_ice_segments/geophysical/bckgrd',
            'gt3r/land_ice_segments/geophysical/bckgrd'],
 'bsnow_conf': ['gt1l/land_ice_segments/geophysical/bsnow_conf',
                'gt1r/land_ice_segments/geophysical/bsnow_conf',
            

#### Example 9: remove all variables+paths associated with a beam
Remove all paths for `gt1l` and `gt3r`

In [22]:
region_a.order_vars.remove(beam_list=['gt1l','gt3r'])
pprint(region_a.order_vars.wanted)

{'atl06_quality_summary': ['gt1r/land_ice_segments/atl06_quality_summary',
                           'gt2l/land_ice_segments/atl06_quality_summary',
                           'gt2r/land_ice_segments/atl06_quality_summary',
                           'gt3l/land_ice_segments/atl06_quality_summary'],
 'atlas_sdp_gps_epoch': ['ancillary_data/atlas_sdp_gps_epoch'],
 'bckgrd': ['gt1r/land_ice_segments/geophysical/bckgrd',
            'gt2l/land_ice_segments/geophysical/bckgrd',
            'gt2r/land_ice_segments/geophysical/bckgrd',
            'gt3l/land_ice_segments/geophysical/bckgrd'],
 'bsnow_conf': ['gt1r/land_ice_segments/geophysical/bsnow_conf',
                'gt2l/land_ice_segments/geophysical/bsnow_conf',
                'gt2r/land_ice_segments/geophysical/bsnow_conf',
                'gt3l/land_ice_segments/geophysical/bsnow_conf'],
 'bsnow_h': ['gt1r/land_ice_segments/geophysical/bsnow_h',
             'gt2l/land_ice_segments/geophysical/bsnow_h',
             'gt2r/land_ice

#### Example 10: generate a default list for the rest of the tutorial
Generate a reasonable variable list prior to download

In [28]:
region_a.order_vars.remove(all=True)
region_a.order_vars.append(defaults=True)
pprint(region_a.order_vars.wanted)

{'atl06_quality_summary': ['gt1l/land_ice_segments/atl06_quality_summary',
                           'gt1r/land_ice_segments/atl06_quality_summary',
                           'gt2l/land_ice_segments/atl06_quality_summary',
                           'gt2r/land_ice_segments/atl06_quality_summary',
                           'gt3l/land_ice_segments/atl06_quality_summary',
                           'gt3r/land_ice_segments/atl06_quality_summary'],
 'atlas_sdp_gps_epoch': ['ancillary_data/atlas_sdp_gps_epoch'],
 'bsnow_conf': ['gt1l/land_ice_segments/geophysical/bsnow_conf',
                'gt1r/land_ice_segments/geophysical/bsnow_conf',
                'gt2l/land_ice_segments/geophysical/bsnow_conf',
                'gt2r/land_ice_segments/geophysical/bsnow_conf',
                'gt3l/land_ice_segments/geophysical/bsnow_conf',
                'gt3r/land_ice_segments/geophysical/bsnow_conf'],
 'bsnow_h': ['gt1l/land_ice_segments/geophysical/bsnow_h',
             'gt1r/land_ice_segment

------------------
### Example Track 2 (Atmosphere - run with ATL09 dataset commented out at the start of the notebook)

#### Example 1: choose variables
Add all `latitude` and `longitude` variables

In [ ]:
region_a.order_vars.append(var_list=['latitude','longitude'])
pprint(region_a.order_vars.wanted)

#### Example 2: specify beams/profiles and variable
Add `latitude` for only `profile_1` and `profile_2`

In [ ]:
region_a.order_vars.remove(all=True)
pprint(region_a.order_vars.wanted)

In [ ]:
var_dict = region_a.order_vars.append(beam_list=['profile_1','profile_2'], var_list=['latitude'])
pprint(region_a.order_vars.wanted)

#### Example 3: add/remove selected beams+variables
Add `latitude` for `profile_3` and remove it for `profile_2`

In [ ]:
region_a.order_vars.append(beam_list=['profile_3'],var_list=['latitude'])
region_a.order_vars.remove(beam_list=['profile_2'], var_list=['latitude'])
pprint(region_a.order_vars.wanted)

#### Example 4: `keyword_list`
Add `latitude` for all profiles and with keyword `low_rate`

In [ ]:
region_a.order_vars.append(var_list=['latitude'],keyword_list=['low_rate'])
pprint(region_a.order_vars.wanted)

#### Example 5: target a specific variable + path
Remove `'profile_1/high_rate/latitude'` (but keep `'profile_3/high_rate/latitude'`)

In [ ]:
region_a.order_vars.remove(beam_list=['profile_1'], var_list=['latitude'], keyword_list=['high_rate'])
pprint(region_a.order_vars.wanted)

#### Example 6: add variables not specific to beams/profiles
Add `rgt` under `orbit_info`.

In [ ]:
region_a.order_vars.append(keyword_list=['orbit_info'],var_list=['rgt'])
pprint(region_a.order_vars.wanted)

#### Example 7: add all variables+paths of a group
In addition to adding specific variables and paths, we can filter all variables with a specific keyword as well. Here, we add all variables under `orbit_info`. Note that paths already in `region_a.order_vars.wanted`, such as `'orbit_info/rgt'`, are not duplicated.

In [ ]:
region_a.order_vars.append(keyword_list=['orbit_info'])
pprint(region_a.order_vars.wanted)

#### Example 8: add all possible values for variables+paths
Append all `longitude` paths and all variables/paths with keyword `high_rate`.
Simlarly to what is shown in Example 4, if you submit only one `append` call as `region_a.order_vars.append(var_list=['longitude'], keyword_list=['high_rate'])` rather than the two `append` calls shown below, you will only add the variable `longitude` and only paths containing `high_rate`, not ALL paths for `longitude` and ANY variables with `high_rate` in their path.

In [ ]:
region_a.order_vars.append(var_list=['longitude'])
region_a.order_vars.append(keyword_list=['high_rate'])
pprint(region_a.order_vars.wanted)

#### Example 9: remove all variables+paths associated with a beam
Remove all paths for `profile_1` and `profile_3`

In [ ]:
region_a.order_vars.remove(beam_list=['profile_1','profile_3'])
pprint(region_a.order_vars.wanted)

#### Example 10: generate a default list for the rest of the tutorial
Generate a reasonable variable list prior to download

In [ ]:
region_a.order_vars.remove(all=True)
region_a.order_vars.append(defaults=True)
pprint(region_a.order_vars.wanted)

## Applying variable subsetting to your order and download

In order to have your wanted variable list included with your order, you must pass it as a keyword argument to the `subsetparams()` attribute or the `order_granules()` or `download_granules()` (which calls `order_granules` under the hood if you have not already placed your order) functions.

In [29]:
region_a.subsetparams(Coverage=region_a.order_vars.wanted)

{'time': '2019-02-22T00:00:00,2019-02-28T23:59:59',
 'Coverage': '/orbit_info/sc_orient,/orbit_info/sc_orient_time,/ancillary_data/atlas_sdp_gps_epoch,/ancillary_data/data_start_utc,/ancillary_data/data_end_utc,/ancillary_data/granule_start_utc,/ancillary_data/granule_end_utc,/ancillary_data/start_delta_time,/ancillary_data/end_delta_time,/gt1l/land_ice_segments/delta_time,/gt1l/residual_histogram/delta_time,/gt1l/segment_quality/delta_time,/gt1r/land_ice_segments/delta_time,/gt1r/residual_histogram/delta_time,/gt1r/segment_quality/delta_time,/gt2l/land_ice_segments/delta_time,/gt2l/residual_histogram/delta_time,/gt2l/segment_quality/delta_time,/gt2r/land_ice_segments/delta_time,/gt2r/residual_histogram/delta_time,/gt2r/segment_quality/delta_time,/gt3l/land_ice_segments/delta_time,/gt3l/residual_histogram/delta_time,/gt3l/segment_quality/delta_time,/gt3r/land_ice_segments/delta_time,/gt3r/residual_histogram/delta_time,/gt3r/segment_quality/delta_time,/quality_assessment/gt1l/delta_time

Or, you can put the `Coverage` parameter directly into `order_granules`:
`region_a.order_granules(Coverage=region_a.order_vars.wanted)`

However, then you cannot view your subset parameters (`region_a.subsetparams`) prior to submitting your order.

In [30]:
region_a.order_granules()# <-- you do not need to include the 'Coverage' kwarg to
                             # order if you have already included it in a call to subsetparams

Total number of data order requests is  1  for  3  granules.
Data request  1  of  1  is submitting to NSIDC
order ID:  5000000800720
Initial status of your order request at NSIDC is:  processing
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order is: complete


In [31]:
path = './download'
region_a.download_granules(path) # <-- you do not need to include the 'Coverage' kwarg to
                             # download if you have already submitted it with your order

Beginning download of zipped output...
Data request 5000000800720 of  1  order(s) is downloaded.
Download complete


### _Why does the subsetter say no matching data was found?_
Sometimes, granules ("files") returned in our initial search end up not containing any data in our specified area of interest. This is because the initial search is completed using summary metadata for a granule. You've likely encountered this before when viewing available imagery online: your spatial search turns up a bunch of images with only a few border or corner pixels, maybe even in no data regions, in your area of interest. Thus, when you go to extract the data from the area you want (i.e. spatially subset it), you don't get any usable data from that image. Note that as of June 2020 this error message produced by the NSIDC subsetter is currently being modified to reflect this behavior as more of an informational warning as opposed to an error. 

## Check the variable list in your downloaded file

Compare the available variables associated with the full dataset relative to those in your downloaded data file.

In [34]:
# put the full filepath to a data file here. You can get this in JupyterHub by navigating to the file,
# right clicking, and selecting copy path. Then you can paste the path in the second set of quotes below.
fn = '/home/jovyan/' + '2020_ICESat-2_Hackweek_Tutorials/06-07.Data_Access/download/processed_ATL06_20190222010344_08490205_003_01.h5'

#### Check the downloaded dataset
Get all `latitude` variables in your downloaded file:

In [35]:
varname = 'latitude'

varlist = []
def IS2h5walk(vname, h5node):
    if isinstance(h5node, h5py.Dataset):
        varlist.append(vname)
    return 

with h5py.File(fn,'r') as h5pt:
    h5pt.visititems(IS2h5walk)
    
for tvar in varlist:
    vpath,vn = os.path.split(tvar)
    if vn==varname: print(tvar) 

gt1l/land_ice_segments/latitude
gt1r/land_ice_segments/latitude
gt2l/land_ice_segments/latitude
gt2r/land_ice_segments/latitude
gt3l/land_ice_segments/latitude
gt3r/land_ice_segments/latitude


#### Compare to the variable paths available in the original data

In [36]:
region_a.order_vars.parse_var_list(region_a.order_vars.avail())[0][varname]

['gt1l/land_ice_segments/latitude',
 'gt1r/land_ice_segments/latitude',
 'gt2l/land_ice_segments/latitude',
 'gt2r/land_ice_segments/latitude',
 'gt3l/land_ice_segments/latitude',
 'gt3r/land_ice_segments/latitude']